In [ ]:
#!pip install obonet
import pandas as pd
from scipy.stats import hypergeom
from statsmodels.stats.multitest import multipletests
import obonet

# obonet could show Go belongs to BP, MF or CC; https://current.geneontology.org/ontology/go.obo

# --- STEP 1: Load Data ---

# Background GO mapping
df = pd.read_csv("Example_use_orth_Zeamays_Maizev5_Pannzer_1004_slimGO_GoList", sep="\t", header=None, names=["Cluster", "Group", "Gene", "GO"])
df = df[["Gene", "GO"]]

In [25]:
df.head()

,Gene,GO
0,Ot_Chr10_00952,GO:0002098
1,Ot_Chr10_00952,GO:0005515
2,Ot_Chr10_00952,GO:0005634
3,Ot_Chr10_00952,GO:0005852
4,Ot_Chr10_00952,GO:0006413


In [27]:
# Query genes
with open("RdDM_Ot_genelist_Up") as f:
    query_genes = set(line.strip() for line in f if line.strip())

# --- STEP 2: Prepare ---

# Unique background genes
background_genes = set(df['Gene'].unique())
bgtotal = len(background_genes)
querytotal = len(query_genes)

# Flag genes in query
df['in_query'] = df['Gene'].isin(query_genes)

# --- STEP 3: Count GO stats ---

go_stats = df.groupby('GO').agg(
    bgitem=('Gene', lambda x: len(set(x))),
    queryitem=('in_query', lambda x: sum(x))
).reset_index()

# Filter: at least 5 gene in query
go_stats = go_stats[go_stats['queryitem'] >= 5]

In [29]:
go_stats.head()

,GO,bgitem,queryitem
379,GO:0003700,881,7
384,GO:0003723,891,6
693,GO:0004497,257,11
966,GO:0005506,315,10
971,GO:0005524,2395,16


In [31]:
# --- STEP 4: Hypergeometric test ---
go_stats['bgtotal'] = bgtotal
go_stats['querytotal'] = querytotal
go_stats['pvalue'] = go_stats.apply(
    lambda row: hypergeom.sf(row['queryitem'] - 1, row['bgtotal'], row['bgitem'], row['querytotal']),
    axis=1
)

# --- STEP 5: FDR correction ---
go_stats['FDR'] = multipletests(go_stats['pvalue'], method='fdr_bh')[1]

# --- STEP 6: Get entries (matching genes) per GO ---
def get_query_genes_for_go(go):
    return df[(df['GO'] == go) & (df['in_query'])]['Gene'].unique()

go_stats['entries'] = go_stats['GO'].apply(
    lambda go: " // ".join(sorted(get_query_genes_for_go(go)))
)


# Keep only GO terms with FDR <= 0.05
go_stats = go_stats[go_stats['FDR'] <= 0.05]

In [33]:
go_stats.head()

,GO,bgitem,queryitem,bgtotal,querytotal,pvalue,FDR,entries
693,GO:0004497,257,11,19102,115,4.375753e-07,0.000010,Ot_Chr10_17026 // Ot_Chr1_19321 // Ot_Chr1_216...
966,GO:0005506,315,10,19102,115,2.084076e-05,0.000092,Ot_Chr1_19321 // Ot_Chr1_21611 // Ot_Chr1_n318...
1679,GO:0008194,138,7,19102,115,2.018118e-05,0.000092,Ot_Chr2_05711 // Ot_Chr2_17543 // Ot_Chr2_n320...
2916,GO:0016491,409,8,19102,115,3.330177e-03,0.010466,Ot_Chr3_11721 // Ot_Chr4_00116 // Ot_Chr4_0036...
2983,GO:0016705,235,9,19102,115,1.256579e-05,0.000092,Ot_Chr1_19321 // Ot_Chr1_21611 // Ot_Chr1_n318...


In [35]:
# --- STEP 7: Load GO terms & namespaces ---
obo_url = "http://purl.obolibrary.org/obo/go.obo"
graph = obonet.read_obo(obo_url)

def get_term_name(go_id):
    return graph.nodes.get(go_id, {}).get('name', 'Unknown')

def get_term_namespace(go_id):
    ns = graph.nodes.get(go_id, {}).get('namespace', '')
    return {'biological_process': 'P', 'molecular_function': 'F', 'cellular_component': 'C'}.get(ns, '?')

go_stats['Term'] = go_stats['GO'].apply(get_term_name)
go_stats['term_type'] = go_stats['GO'].apply(get_term_namespace)

# --- STEP 8: Rearrange & Export ---
go_stats = go_stats[['GO', 'term_type', 'Term', 'queryitem', 'querytotal', 'bgitem', 'bgtotal', 'pvalue', 'FDR', 'entries']]
go_stats.columns = ["GO_acc", "term_type", "Term", "queryitem", "querytotal", "bgitem", "bgtotal", "pvalue", "FDR", "entries"]

In [36]:
go_stats.head()

,GO_acc,term_type,Term,queryitem,querytotal,bgitem,bgtotal,pvalue,FDR,entries
693,GO:0004497,F,monooxygenase activity,11,115,257,19102,4.375753e-07,0.000010,Ot_Chr10_17026 // Ot_Chr1_19321 // Ot_Chr1_216...
966,GO:0005506,F,iron ion binding,10,115,315,19102,2.084076e-05,0.000092,Ot_Chr1_19321 // Ot_Chr1_21611 // Ot_Chr1_n318...
1679,GO:0008194,F,UDP-glycosyltransferase activity,7,115,138,19102,2.018118e-05,0.000092,Ot_Chr2_05711 // Ot_Chr2_17543 // Ot_Chr2_n320...
2916,GO:0016491,F,oxidoreductase activity,8,115,409,19102,3.330177e-03,0.010466,Ot_Chr3_11721 // Ot_Chr4_00116 // Ot_Chr4_0036...
2983,GO:0016705,F,"oxidoreductase activity, acting on paired dono...",9,115,235,19102,1.256579e-05,0.000092,Ot_Chr1_19321 // Ot_Chr1_21611 // Ot_Chr1_n318...


In [39]:
# Save
go_stats.sort_values("FDR").to_csv("result_GOTerm_Ot.txt", sep="\t", index=False)